In [1]:
#Importaciones y configuración inicial
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors, DenseVector
import os
import shutil
import re
import math
import ast
from itertools import combinations
import nltk

# Descargar stopwords de NLTK y de ahí las usamos para hacer nuestro paso 3
try:
    nltk.data.find('corpora/stopwords')
    print("Ya están descargadas")
except LookupError:
    nltk.download('stopwords', quiet=True)
    print("Ya se descargaron las stopwords")

# Inicializar Spark
spark = SparkSession.builder \
    .appName("ProyectoSparkie-ProyectoFinal") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

Ya están descargadas


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/09 00:46:16 WARN Utils: Your hostname, DESKTOP-6OAF9F9, resolves to a loopback address: 127.0.1.1; using 172.28.59.214 instead (on interface eth0)
25/12/09 00:46:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/09 00:47:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#PASO 3 - Cargar libros ya sin los encabezados del Gunterberg y preparar datos
#======= Variables:
# rdd_libros: RDD donde guardamos los libros y en la que trabajaremos
# num_libros: Guardamos cuantos libros son y usarlo en calculos posteriores
# rdd_lower: RDD de los libros transormados en minusculas
# rdd_clean: RDD donde guardamos las palabras minusculas para quitarles los caracteres y simbolos innecesarios
# rdd_tokens: RDD donde ya están los textos tokenizados 

# Cargar los libros desde la carpeta raw en la que guardamos nuestros libros (Cambiar esta parte si cambiamos de lugar los libros)
base_path = os.path.abspath("../data/raw")
rdd_libros = sc.wholeTextFiles(base_path)
num_libros = rdd_libros.count()

print(f"Libros cargados: {num_libros}") # Para confirmar que la variable funciona y cuantos son. Lo podemos eliminar.

# Convertir a minúsculas: 
rdd_lower = rdd_libros.map(lambda x: (x[0], x[1].lower()))

# Función para limpiar caracteres y símbolos innecesarios
def limpiar(texto):
    return re.sub(r"[^a-z0-9\s]", " ", texto)

rdd_clean = rdd_lower.map(lambda x: (x[0], limpiar(x[1])))

# Tokenización
rdd_tokens = rdd_clean.flatMap(lambda x: [(x[0], t) for t in x[1].split()])

Libros cargados: 99


In [3]:
#Filtrar stopwords y preparar datos
# ========= Variables:
# stop = Lista de stopwords sacadas de librería nltk
# stop_b = broadcast de stop
# rdd_filtrado = RDD con TODOS los filtros aplicados.
# rdd_doc_token = Nombres de los documentos

from nltk.corpus import stopwords

# Cargar y broadcast stopwords
stop = set(stopwords.words("english"))
stop_b = sc.broadcast(stop)

# Función para quitar palabras menores a 3
def es_valida(palabra):
    return palabra not in stop_b.value and len(palabra) > 2

# Filtrar tokens
rdd_filtrado = rdd_tokens.filter(lambda x: es_valida(x[1]))

# Extraer nombre del documento
def docname(path):
    return os.path.basename(path)

rdd_doc_token = rdd_filtrado.map(lambda x: (docname(x[0]), x[1]))

In [ ]:
# Crear vocabulario y guardar Vocabulario por documento (palabra única por documento)
# ======== Variables:
# rdd_vocab = rdd de palabras unicas por documento
# output_vocab = string de la ubicación de nuestro vocabulario y donde guardaremos el rdd_vocab
# vocab_global = Vocabulario global de todos los documentos
# output_global = string con la ubicación donde guardaremos output_global

rdd_vocab = rdd_doc_token.distinct()

# Guardar vocabulario por documento
output_vocab = "../data/processed/vocabulario_rdd" # ubicación de nuestros vocabularios procesados, cambiar si movemos la ubicación
if os.path.exists(output_vocab):
    shutil.rmtree(output_vocab)
rdd_vocab.saveAsTextFile(output_vocab)
print(f"\nVocabulario guardado en: {output_vocab}") # -> Mensaje para confirmar

# Vocabulario global (todas las palabras únicas del corpus)
vocab_global = rdd_doc_token.map(lambda x: x[1]).distinct()
print(f"Vocabulario global: {vocab_global.count()} palabras únicas") #Confirmamos

# Guardar vocabulario global
output_global = "../data/processed/vocabulario_unico_rdd"
if os.path.exists(output_global):
    shutil.rmtree(output_global)
vocab_global.saveAsTextFile(output_global)
print(f"Vocabulario global guardado en: {output_global}") #Confirmamos

25/12/09 01:00:45 WARN Executor: Issue communicating with driver in heartbeater]
org.apache.spark.rpc.RpcTimeoutException: Future timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1292)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:307)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.scal


Vocabulario guardado en: ../data/processed/vocabulario_rdd


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 2) / 2]
Traceback (most recent call last):
  File "/home/tlaloc/proyecto-sparkie/venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tlaloc/proyecto-sparkie/venv/lib/python3.12/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
[Stage 3:>                                                          (0 + 2) / 2]

KeyboardInterrupt: 

In [ ]:
# Calcular frecuencias de palabras
# ============ Variables:
# rdd_pairs = Duplas en formato ((documento, palabra), 1)
# rdd_freq = rdd reducida para saber la frecuencia
# output_freq = String de la ubicación donde guardaremos rdd_freq

# Crear duplas ((documento, palabra), 1)
rdd_pairs = rdd_doc_token.map(lambda x: ((x[0], x[1]), 1))

# Reducir por clave para contar frecuencias
rdd_freq = rdd_pairs.reduceByKey(lambda a, b: a + b)

# Esto lo usamos para saber si está bien así
#print(f"Frecuencias calculadas: {rdd_freq.count()} pares (documento, palabra)")
#print("\n  Primeros 5 pares con frecuencias:")
#for item in rdd_freq.take(5):
#    print(f"    {item[0]}: {item[1]} apariciones")

# Guardar frecuencias
output_freq = "../data/processed/frecuencias_rdd"
if os.path.exists(output_freq):
    shutil.rmtree(output_freq)
rdd_freq.saveAsTextFile(output_freq)

print(f"\nFrecuencias guardadas en: {output_freq}")

In [ ]:
#Aquí inicia nuestro PASO 4 de crear matriz de similitud
# ======== Variables:
# rdd_parsed = tublas de frecuencias en (doc, palabra, freq) usando de referencia rdd_freq
# doc_totals = total palabras por documento
# rdd_doc_word_freq = rdd ((documento, palabra), frecuencia)
# rdd_with_totals = joins con totales de rdd_doc_word_freq
# rdd_tf = rdd con las TF

# Función que convierte tupla ((doc, palabra), freq) a (doc, palabra, freq)
def parse_freq_tuple(x):
    return (x[0][0], x[0][1], x[1])

# Parsear frecuencias 
rdd_parsed = rdd_freq.map(parse_freq_tuple)

# Calcular total de palabras por documento
doc_totals = rdd_parsed.map(lambda x: (x[0], x[2])) \
                       .reduceByKey(lambda a, b: a + b)

# print(f"Total de palabras calculado para {doc_totals.count()} documentos")

# Calcular TF con formula: freq / total_palabras_doc

rdd_doc_word_freq = rdd_parsed.map(lambda x: ((x[0], x[1]), x[2]))

# Join con totales
rdd_with_totals = rdd_doc_word_freq.map(lambda x: (x[0][0], (x[0][1], x[1]))) \
                                   .join(doc_totals) \
                                   .map(lambda x: ((x[0], x[1][0][0]), (x[1][0][1], x[1][1])))

# Calcular TF
rdd_tf = rdd_with_totals.map(lambda x: (x[0], x[1][0] / x[1][1]))

print("TF (Term Frequency) calculado")

In [ ]:
# Aquí se calcula IDF
# ====== Variables
# total_docs = Documentos totales (Lo calculamos aopartir de doc_totals)
# rdd_word_doc_count = Cantidad de documentos en los que aparece una palabra
# rdd_idf = rdd con las idf calculadas

# Contar total de documentos
total_docs = doc_totals.count()
print(f"Total de documentos en el corpus: {total_docs}")

# Contar en cuántos documentos aparece cada palabra
rdd_word_doc_count = rdd_parsed.map(lambda x: (x[1], x[0])) \
                               .distinct() \
                               .map(lambda x: (x[0], 1)) \
                               .reduceByKey(lambda a, b: a + b)

# Calcular IDF: log(total_docs / docs_con_palabra)
rdd_idf = rdd_word_doc_count.map(lambda x: (x[0], math.log(total_docs / x[1])))

#print("IDF calculado")
#print(f" Total de palabras únicas con IDF: {rdd_idf.count()}")
#print(f" ejemplo de idf para ver como quedó: {rdd_idf.first()}")

In [ ]:
# En esta calculamos TF-IDF y ya guardar
# rdd_tf_prep -> (palabra, (doc, tf)) = rdd preparando para hacer el calculo
# rdd_tf_idf -> (palabra, ((doc, tf), idf)) = TF-IDF calculados
# output_tfidf = String de ubicación de tfidf

# Preparar TF: ((doc, palabra), tf) -> (palabra, (doc, tf))
rdd_tf_prep = rdd_tf.map(lambda x: (x[0][1], (x[0][0], x[1])))

# Join TF con IDF: (palabra, ((doc, tf), idf))
rdd_tf_idf = rdd_tf_prep.join(rdd_idf) \
                        .map(lambda x: ((x[1][0][0], x[0]), x[1][0][1] * x[1][1]))

#print("TF-IDF calculado")
#print(f" Total de valores TF-IDF: {rdd_tf_idf.count()}")
#print(f" Uno de ejemplo: {rdd_tf_idf.first()}")

# Guardar TF-IDF
output_tfidf = "../data/processed/tfidf_rdd"

if os.path.exists(output_tfidf):
    shutil.rmtree(output_tfidf)
rdd_tf_idf.saveAsTextFile(output_tfidf)
print(f"TF-IDF guardado en: {output_tfidf}")

In [ ]:
#Creamos vectores TF-IDF por documento
# ================= == Variables
# vocab_global_indexed = Vocabulario global por indice
# vocab_dict = diccionario de datos
# vocab_size = Contar numero de palabras

# Crear vocabulario global con índices
vocab_global_indexed = rdd_tf_idf.map(lambda x: x[0][1]).distinct().zipWithIndex()
vocab_dict = vocab_global_indexed.collectAsMap()
vocab_size = len(vocab_dict)

#print(f"Hay {vocab_size} palabras únicas")

# Convertir TF-IDF a formato (doc, [(índice, tfidf), ...])
rdd_doc_vectors = rdd_tf_idf.map(lambda x: (x[0][0], (vocab_dict[x[0][1]], x[1]))) \
                            .groupByKey() \
                            .map(lambda x: (x[0], list(x[1])))


In [ ]:
# Función de similitud coseno
def cosine_similarity_sparse(vec1, vec2):
    """
    Calcula similitud coseno entre dos vectores dispersos
    Parámetros:
    - vec1, vec2: lista de tuplas [(índice, valor), ...]
    Retorna:
    - similitud coseno con valores de (0 a 1)
    """
    # Convertir a diccionarios para búsqueda rápida
    dict1 = dict(vec1)
    dict2 = dict(vec2)
    
    # Producto punto
    dot_product = sum(dict1.get(idx, 0) * dict2.get(idx, 0) 
                      for idx in set(dict1.keys()) | set(dict2.keys()))
    
    # Normas
    norm1 = math.sqrt(sum(v**2 for v in dict1.values()))
    norm2 = math.sqrt(sum(v**2 for v in dict2.values()))
    
    if norm1 == 0 or norm2 == 0:
        return 0.0
    
    return dot_product / (norm1 * norm2)

#print("Función de similitud coseno definida")

In [ ]:
# Aquí se calcula matriz de similitud
# ========== Variables:
# doc_vectors_list = lista de vectores por documento
# 
# 

# Recolectar vectores
doc_vectors_list = rdd_doc_vectors.collect()
doc_names = [doc[0] for doc in doc_vectors_list]
total_pairs = len(doc_names) * (len(doc_names) - 1) // 2

#print(f"Total de pares a calcular: {total_pairs}")

# Calcular similitudes
similarities = []
for i, (doc1, vec1) in enumerate(doc_vectors_list):
    for doc2, vec2 in doc_vectors_list[i+1:]:
        sim = cosine_similarity_sparse(vec1, vec2)
        similarities.append((doc1, doc2, sim))
    
    if (i + 1) % 10 == 0:
        print(f"  Procesados: {i+1}/{len(doc_vectors_list)} documentos...")

# Convertir a RDD
rdd_similarities = sc.parallelize(similarities)

print(f"\nSimilitudes calculadas: {rdd_similarities.count()} pares")

In [ ]:
# Guardamos matriz de similitud
output_sim = "../data/processed/similarity_matrix_rdd"
if os.path.exists(output_sim):
    shutil.rmtree(output_sim)

rdd_similarities.saveAsTextFile(output_sim)
print(f"Matriz de similitudes guardada en: {output_sim}")

# Extraer lista de libros disponibles para usar en funciones posteriores
libros_disponibles = rdd_similarities.flatMap(lambda x: [x[0], x[1]]).distinct().collect()
print(f"Lista de libros disponibles: {len(libros_disponibles)} libros")

In [ ]:
# Aquí iniciamos nuestro PASO 5 - Función de búsqueda de libros

def buscar_libro(nombre_libro, libros_disponibles):
    """
    Busca un libro en la colección
    Parámetros:
    - nombre_libro: nombre del libro (búsqueda parcial permitida)
    - libros_disponibles: lista de todos los libros
    Retorna:
    - nombre completo del libro si existe
    - None si no existe o hay ambigüedad
    """
    nombre_lower = nombre_libro.lower()
    
    # Búsqueda exacta
    for libro in libros_disponibles:
        if libro.lower() == nombre_lower:
            return libro
    
    # Búsqueda parcial
    coincidencias = [libro for libro in libros_disponibles 
                     if nombre_lower in libro.lower()]
    
    if len(coincidencias) == 1:
        print(f"Libro encontrado: {coincidencias[0]}")
        return coincidencias[0]
    elif len(coincidencias) > 1:
        print(f"Se encontraron {len(coincidencias)} libros que coinciden:")
        for i, libro in enumerate(coincidencias[:5], 1):
            print(f"   {i}. {libro}")
        if len(coincidencias) > 5:
            print(f"   ... y {len(coincidencias) - 5} más")
        print("\n Por favor, sea más específico con el nombre :c")
        return None
    else:
        print(f"Error: El libro '{nombre_libro}' no existe en la colección")
        print("\n Sugerencias (libros similares):")
        palabras = nombre_lower.split()
        sugerencias = []
        for libro in libros_disponibles:
            libro_lower = libro.lower()
            if any(palabra in libro_lower for palabra in palabras):
                sugerencias.append(libro)
        
        for i, libro in enumerate(sugerencias[:5], 1):
            print(f"   {i}. {libro}")
        
        return None

# print("Función de búsqueda de libros definida")

In [ ]:
# Paso 5

def recomendar_libros(libro_favorito, n):
    """
    Recomienda N libros más similares al libro favorito del usuario.
    
    Parámetros:
    - libro_favorito: nombre del libro (str)
    - n: cantidad de libros a recomendar (int, default=5)
    
    Imprime:
    - N libros recomendados con su similitud
    """
    # Validar que el libro existe
    libro_completo = buscar_libro(libro_favorito, libros_disponibles)
    
    if libro_completo is None:
        return
    
    print(f"\nBuscando libros similares a: {libro_completo}")
    print(f"   Cantidad solicitada: {n} recomendaciones\n")
    
    # Filtrar similitudes donde aparece el libro
    similitudes_libro = rdd_similarities.filter(
        lambda x: x[0] == libro_completo or x[1] == libro_completo
    )
    
    # Reformatear: (libro_recomendado, similitud)
    recomendaciones = similitudes_libro.map(
        lambda x: (x[1], x[2]) if x[0] == libro_completo else (x[0], x[2])
    )
    
    # Ordenar por similitud mayor a menor
    top_recomendaciones = recomendaciones.sortBy(lambda x: x[1], ascending=False).take(n)
    
    # Imprimir resultados
    print("Recomendaciones:")
    for i, (libro, similitud) in enumerate(top_recomendaciones, 1):
        print(f"   {i}. [{similitud:.4f}] {libro}")

#print("Función de recomendación definida")

In [ ]:
# PASO 6 - Función de palabras descriptivas

def palabras_descriptivas(libro, M):
    """
    Muestra las M palabras más representativas del libro.
    Basado en frecuencia de aparición.
    
    Parámetros:
    - libro: nombre del libro (str)
    - M: cantidad de palabras a mostrar (int, default=10)
    
    Imprime:
    - M palabras más frecuentes con su número de apariciones
    """
    libro_ok = buscar_libro(libro, libros_disponibles)
    
    if libro_ok is None:
        return
    
    print(f"\n Palabras clave de: {libro_ok}\n")
    
    # Filtrar solo ese documento del RDD de frecuencias
    palabras = (rdd_freq
        .filter(lambda x: x[0][0] == libro_ok)  # x[0] = (doc, palabra)
        .map(lambda x: (x[0][1], x[1]))         # (palabra, freq)
        .sortBy(lambda x: x[1], ascending=False)
        .take(M)
    )
    
    # Imprimir resultados
    print("- Palabras que describen el libro:\n")
    for i, (palabra, freq) in enumerate(palabras, 1):
        print(f"   {i}. {palabra:20} → {freq:5} apariciones")

#print("Función de palabras descriptivas definida")

In [ ]:
# Ejemplos
recomendar_libros("The_Expedition_of_Humphry_Clinker", n=5)
palabras_descriptivas("The Odyssey", M=15)
recomendar_libros("Harry Potter", n=5)


In [ ]:
sc.stop()